# 🦌 SVG Defs 🏹

This notebook demonstrates how to define custom symbols and attach them to the ipyelk
diagram.

In [ ]:
import json
import pathlib
from dataclasses import asdict

import importnb
import ipywidgets
import networkx as nx
import traitlets
from IPython.display import JSON, SVG

import ipyelk
import ipyelk.diagram.defs.connectors as conn
import ipyelk.diagram.elk_export
import ipyelk.nx
import ipyelk.tools
from ipyelk import Elk
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.defs import shapes
from ipyelk.diagram.defs.svg import (
    Circle,
    ConnectorDef,
    Def,
    Ellipse,
    Path,
    Point,
    RawSVG,
    Rect,
)
from ipyelk.diagram.elk_model import ElkLabel, ElkNode, ElkPort

# Defining some custom arrows

The `ConnectorDef` class allows a list of svg children to be specified. These children
define the custom svg shape. The `ConnectorDef` also takes two other parameters,
`correction` and `offset`. The `correction` parameter adjusts the anchor endpoint and
the `offset` moves the endpoint of the calculated path.

In [ ]:
arrow = ConnectorDef(
    children=[
        Path(
            segments=[
                Point(5, -5),
                Point(0, 0),
                Point(5, 5),
            ],
            closed=False,
        ),
    ],
    correction=Point(-1, 0),  # moves the arrow endpoint nearer the outside of the node
    offset={"x": -5, "y": 0},  # shortens the path
)

# this connector is composed of two svg shapes
circle = ConnectorDef(
    children=[
        Path(
            segments=[Point(5, -5), Point(5, 5)],
            closed=False,
        ),
        Circle(radius=4, position={"x": 4, "y": 0}),
    ],
)

# Shape Library

# Wiring up ElkDiagram with def library

In [ ]:
def use_simple_custom_arrows():
    # build graph
    g = nx.Graph()

    g.add_edge(
        "n1",
        "n2",
        properties={
            "start": "arrow",  # key of the shape in the def dictionary
            "end": "arrow2",  # key of the shape in the def dictionary
        },
    )

    # configure app
    app = Elk(transformer=ipyelk.nx.XELK(source=(g, None)), layout={"height": "100%"})

    app.diagram.defs = {
        "arrow": arrow,
        "arrow2": circle,
    }
    return app


if __name__ == "__main__":
    app = use_simple_custom_arrows()
    display(app)

# Shape functions

In [ ]:
def Interface(r=6):
    return Def(
        children=[
            Rect(width=r, height=r),
            Path(
                segments=[
                    Point(1, 0.25 * r),
                    Point(r - 0.25 * r, r / 2),
                    Point(1, r - 0.25 * r),
                ]
            ),
        ]
    )


def Activity(r=6):
    return Def(children=[Ellipse(rx=r, ry=r / 2, position=Point(r, r / 2))])


shape_library = {
    # connector shapes
    "composition": conn.Rhomb(r=4),
    "aggregation": conn.Rhomb(r=4),
    "containment": conn.Containment(r=4),
    "arrow": conn.StraightArrow(r=4),
    # node shape,
    "final_state": shapes.DoubleCircle(r=6),
    # port shape
    "interface_port": Interface(r=5),
}

In [ ]:
def example_with_ports():
    # build graph
    g = nx.Graph()

    g.add_node("n1")
    g.add_node(
        "n4",
        labels=[""],  # no label will be used
        properties={
            "use": "final_state",  # key of the shape in the def dictionary
        },
    )

    g.add_node(
        "n2",
        ports=[
            {
                "id": "n2.x",
                "properties": {
                    "use": "interface_port"  # key of the shape in the def dictionary
                },
            },
            {
                "id": "n2.y",
                "properties": {
                    "use": "interface_port"  # key of the shape in the def dictionary
                },
            },
        ],
    )

    g.add_edge(
        "n1",
        "n2",
        sourcePort="x",
        targetPort="x",
        properties={
            "start": "containment",
            "end": "arrow",
        },
    )

    g.add_edge(
        "n1",
        "n3",
        properties={
            "start": "aggregation",
            "end": "arrow",
        },
    )

    g.add_edge(
        "n2",
        "n3",
        properties={
            "start": "composition",
        },
    )
    g.add_edge(
        "n4",
        "n1",
    )
    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(source=(g, None)),
        layout={"height": "100%"},
        style={
            " .elkarrow.aggregation": {"fill": "var(--jp-elk-node-stroke)"},
            " .final_state > circle:nth-child(2)": {
                "fill": "var(--jp-elk-node-stroke)"
            },
        },
    )

    app.diagram.defs = shape_library

    return app


if __name__ == "__main__":
    app = example_with_ports()
    display(app)

In [ ]:
def exporter_example(app):
    import ipywidgets as W

    exporter = ipyelk.diagram.elk_export.ElkExporter(
        diagram=app.diagram, app=app, strip_ids=False
    )
    out = W.Output()

    def update(change=None):
        out.clear_output()
        with out:
            display(SVG(exporter.value))

    exporter.observe(update, "value")
    update()
    return out


if __name__ == "__main__":
    app = example_with_ports()
    out = exporter_example(app)
    display(app)
    display(out)

# Shape Functions

In [ ]:
def shape_library_diagram(shape_library, style):
    # configure app
    app = Elk(transformer=ipyelk.nx.XELK(), layout={"height": "100%"}, style=style)

    app.diagram.defs = shape_library

    # build graph
    g = nx.Graph()

    for key in shape_library.keys():
        g.add_node("mark_" + key, properties={"use": key})

    app.transformer.source = (g, None)

    return app


if __name__ == "__main__":
    app = shape_library_diagram(
        shape_library={
            "composition": conn.Rhomb(r=4),
            "aggregation": conn.Rhomb(r=4),
            "containment": conn.Containment(r=4),
            "arrow": conn.StraightArrow(r=4),
            "inheritance": conn.StraightArrow(r=4, closed=True),
            "space": conn.Space(r=2),
            # activity nodes,
            "decision": shapes.Diamond(r=12),
            "merge": shapes.Diamond(r=12),
            "final_state": shapes.DoubleCircle(r=6),
            "start_state": shapes.SingleCircle(r=6),
            "exit_state": shapes.XCircle(r=6),
            # port interface
            "interface_port": Interface(r=5),
            "activity": Activity(r=20),
        },
        style={
            " .aggregation": {"fill": "var(--jp-elk-node-stroke)"},
            " .final_state > circle:nth-child(2)": {
                "fill": "var(--jp-elk-node-stroke)"
            },
        },
    )
    display(app)